# 清洗LMSYS-Chat-1M数据
- 语言为英文
- 对话轮次大于2
- 对对话进行编号
- 记录轮次以及对话位置
- 只有gpt的对话

In [2]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("lmsys/lmsys-chat-1m")

Using the latest cached version of the dataset since lmsys/lmsys-chat-1m couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /Users/zhuruichen/.cache/huggingface/datasets/lmsys___lmsys-chat-1m/default/0.0.0/200748d9d3cddcc9d782887541057aca0b18c5da (last modified on Thu Mar 20 18:01:23 2025).
Found the latest cached dataset configuration 'default' at /Users/zhuruichen/.cache/huggingface/datasets/lmsys___lmsys-chat-1m/default/0.0.0/200748d9d3cddcc9d782887541057aca0b18c5da (last modified on Thu Mar 20 18:01:23 2025).


In [3]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['conversation_id', 'model', 'conversation', 'turn', 'language', 'openai_moderation', 'redacted'],
        num_rows: 1000000
    })
})


In [4]:
first_sample = ds['train'][1:2]
print(first_sample)

{'conversation_id': ['1e230e55efea4edab95db9cb87f6a9cb'], 'model': ['vicuna-13b'], 'conversation': [[{'content': "Beside OFAC's selective sanction that target the listed individiuals and entities, please elaborate on the other types of US's sanctions, for example, comprehensive and sectoral sanctions. Please be detailed as much as possible", 'role': 'user'}, {'content': "The United States has a number of different types of sanctions that it can use to achieve its foreign policy goals, including both selective and comprehensive sanctions.\n\nSelective sanctions are targeted at specific individuals or entities that are believed to be engaged in activities that are contrary to US interests. These sanctions can take a variety of forms, including asset freezes, travel bans, and restrictions on financial transactions. The Office of Foreign Assets Control (OFAC) is the US government agency responsible for implementing and enforcing these types of sanctions.\n\nComprehensive sanctions, on the 

In [5]:
import pandas as pd

# Filter the dataset
filtered_ds = ds['train'].filter(
    lambda x: (x['language'] == 'English') and 
              ('gpt' in x['model'].lower()) and 
              len(x['conversation']) > 2  # Ensuring more than 1 turn
)

# Process conversations
conversations_data = []

for item in filtered_ds:
    conversation_id = item['conversation_id']
    model = item['model']
    conversation = item['conversation']
    
    current_turn = 0
    for i, message in enumerate(conversation):
        if not isinstance(message, dict) or 'role' not in message or 'content' not in message:
            continue
        
        role = message['role']
        content = message['content']
        
        # Increment turn when we see a user message (except first one if it's system)
        if role == 'user' and i > 0:
            current_turn += 1
        
        # Initialize id_mapping if it doesn't exist yet
        if 'id_mapping' not in globals():
            globals()['id_mapping'] = {}
        
        # Convert conversation_id to string for consistency
        conv_id_str = str(conversation_id[0]) if isinstance(conversation_id, list) else str(conversation_id)
        
        # Assign new sequential ID if this conversation hasn't been seen before
        if conv_id_str not in globals()['id_mapping']:
            globals()['id_mapping'][conv_id_str] = len(globals()['id_mapping'])
        
        conversations_data.append({
            'conversation_id': globals()['id_mapping'][conv_id_str],
            'model': model,
            'role': role,
            'content': content,
            'turn': current_turn,
            'position': i
        })

# Create DataFrame
df = pd.DataFrame(conversations_data)

# Save to CSV
df.to_csv('../cleaned_data/lmsys_processed.csv', index=False)

print(f"Processed data saved to lmsys_processed.csv")
print(f"Total conversations: {len(filtered_ds)}")
print(f"Total messages: {len(df)}")

# Processed data saved to lmsys_processed.csv
# Total conversations: 5785
# Total messages: 45362

Processed data saved to lmsys_processed.csv
Total conversations: 5785
Total messages: 45362
